##  =====================================================
##  JOB: GOLD - Metadados Terrabrasilis (Camada Gold)
##  =====================================================
##  Cria a tabela Gold com descrições (comments) no Unity Catalog
##  =====================================================

In [0]:
from pyspark.sql import functions as F

In [0]:
# ================================================================
# Parâmetro recebido via Databricks Job
# ================================================================
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")
dbutils.widgets.text("schema_out", "")
dbutils.widgets.text("table", "")
dbutils.widgets.text("table_out", "")
dbutils.widgets.text("data_ref_carga", "")
catalog         = dbutils.widgets.get("catalog")
schema          = dbutils.widgets.get("schema")
schema_out      = dbutils.widgets.get("schema_out")
table           = dbutils.widgets.get("table")
table_out       = dbutils.widgets.get("table_out")
data_ref_carga  = dbutils.widgets.get("data_ref_carga")

In [0]:
silver_table = f"{catalog}.{schema}.{table}"
gold_table   = f"{catalog}.{schema_out}.{table_out}"

In [0]:
print(f"🔹 Iniciando processamento GOLD\nOrigem: {silver_table}\nDestino: {gold_table}")

In [0]:
# -----------------------------------------------------
# Leitura da camada Silver
# -----------------------------------------------------
df_silver = spark.table(silver_table) \
                 .filter(F.col("data_ref_carga") == data_ref_carga)

In [0]:
# Verificar se está vazio
if df_silver.limit(1).count() == 0:
    print(f"Nenhum dado encontrado para data_ref_carga={data_ref_carga}. Encerrando job.")
    dbutils.notebook.exit("Silver vazia")

In [0]:
# --- Construir GOLD com campos corretos ---
df_gold = df_silver.select(
    "data_ref_carga",
    F.col("title").alias("title"),
    F.col("publication_date").alias("publication_date"),
    F.col("purpose").alias("purpose"),
    F.col("abstract").alias("abstract"),
    F.col("license_info").alias("license_info"),
    F.expr("filter(emails, x -> x != '')[0]").alias("email_principal"),
    F.concat_ws(", ", "keywords").alias("keywords_str"),
    F.to_json(F.struct(
        F.col("westBoundLongitude").alias("west"),
        F.col("southBoundLatitude").alias("south"),
        F.col("eastBoundLongitude").alias("east"),
        F.col("northBoundLatitude").alias("north")
    )).alias("geo_coord_json"),
    F.col("graphic_file"),
    F.col("graphic_description"),
    F.expr("transform(regexp_extract_all(abstract,'(https?://[^\\s]+)',0), x -> x)").alias("urls"),
    F.current_timestamp().alias("data_processamento")
)

In [0]:
# -----------------------------------------------------
# Seleção e renomeação final
# -----------------------------------------------------
df_gold = df_gold.select(
    F.col("title"),
    F.col("publication_date"),
    F.col("purpose"),
    F.col("abstract"),
    F.col("license_info"),
    F.col("email_principal"),
    F.col("keywords_str"),
    F.col("geo_coord_json"),
    F.col("graphic_file"),
    F.col("graphic_description"),
    F.col("urls"),
    F.col("data_processamento"),
    F.col("data_ref_carga")
)

In [0]:
# --- Gravar dados na GOLD ---
df_gold.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("data_ref_carga") \
    .saveAsTable(gold_table)

print(f"✅ GOLD criado e dados inseridos para a partição {data_ref_carga}")

In [0]:
#Verificacao para inserir cometario as colunas na primeira vez
df_gold = spark.table(gold_table) \
                 .filter(F.col("data_ref_carga") == data_ref_carga)

if df_gold.limit(1).count() == 1:
    # -----------------------------------------------------
    # Aplicar descrições (comments) às colunas
    # -----------------------------------------------------
    col_descriptions = {
        "data_ref_carga": "Data de referência da carga dos dados (partição)",
        "title": "Título do dataset conforme metadado original",
        "publication_date": "Data de publicação original do dataset",
        "purpose": "Finalidade ou objetivo do dataset",
        "abstract": "Resumo do conteúdo do metadado (até 400 caracteres)",
        "license_info": "Informações sobre licença de uso do dataset",
        "email_principal": "Contato principal (e-mail do autor ou responsável técnico)",
        "keywords_str": "Lista de palavras-chave associadas ao dataset, concatenadas",
        "geo_coord_json": "Coordenadas geográficas em formato JSON (west, south, east, north)",
        "graphic_file": "URL de arquivo gráfico de referência (imagem ou miniatura do dataset)",
        "graphic_description": "Descrição do gráfico ou imagem ilustrativa associada ao dataset",
        "urls": "Lista de URLs encontradas no campo abstract (documentação, referências)",
        "data_processamento": "Timestamp de processamento e carga da camada Gold"
    }

    # Adiciona os comentários no catálogo
    for col, desc in col_descriptions.items():
        spark.sql(f"ALTER TABLE {gold_table} ALTER COLUMN {col} COMMENT '{desc}'")

    print("📝 Comentários aplicados com sucesso às colunas.")

    # -----------------------------------------------------
    # Descrição geral da tabela
    # -----------------------------------------------------
    table_comment = """
    Metadados extraídos do portal Terrabrasilis (INPE),Polígonos dos novos limites dos biomas, para uso auxiliar, provenientes do dado original composto pelos limites dos Biomas do Brasil.
    """

    spark.sql(f"COMMENT ON TABLE {gold_table} IS '{table_comment.strip()}'")

    print("📘 Descrição geral da tabela GOLD registrada com sucesso.")